In [1]:
import cftime
import glob
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
from scipy import stats

import mission_tools

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
mission_list = glob.glob('../microSWIFT_data/final_dataset/mission_*.nc')

hs_all = []
tm_all = []
mean_dir_all = []
xs_gamma_loc_all = []
water_level_all = []
mission_number = []
mission_break_depth_all = []
mission_time_all = []
mission_spectra_all = []
mission_dir_spectra_all = []
freq_all = []
dir_bins_all = []

gamma = 0.35
awac_file = '../microSWIFT_data/FRFdata/FRF-ocean_waves_awac-6m_202110.nc'
bathy_file = '../microSWIFT_data/FRFdata/FRF_geomorphology_DEMs_surveyDEM_20211021.nc'
bathy_dataset = nc.Dataset(bathy_file)
xFRF_grid, yFRF_grid = np.meshgrid(bathy_dataset['xFRF'][:],bathy_dataset['yFRF'][:])
bathy = bathy_dataset['elevation'][0,:,:]
waterlevel_file = '../microSWIFT_data/FRFdata/FRF-ocean_waterlevel_eopNoaaTide_202110.nc'
waterlevel_dataset = nc.Dataset(waterlevel_file)
waterlevel = waterlevel_dataset['waterLevel'][:]
waterlevel_time = waterlevel_dataset['time'][:]

for mission_nc in mission_list:
    mission_dataset = nc.Dataset(mission_nc, mode='r')

    mission_time = np.median(mission_dataset['time'][:])

    # Compute depth that waves should be breaking in 
    mission_hs = mission_tools.closest_awac_sig_wave_height(mission_time, awac_file)
    mission_tm = mission_tools.closest_awac_Tm(mission_time, awac_file)
    mission_mean_dir = mission_tools.closest_awac_dir(mission_time, awac_file)
    mission_spectra, freq = mission_tools.closest_awac_spectra(mission_time, awac_file)
    mission_dir_spectra, _, dirs = mission_tools.closest_awac_dir_spectra(mission_time, awac_file)

    # Compute 
    mission_break_depth = mission_hs / gamma
    mission_xs_gamma_location = np.interp(-mission_break_depth, np.flip(np.nanmean(bathy, axis=0)), np.flip(bathy_dataset['xFRF'][:]))

    # Save all values
    hs_all.append(mission_hs)
    tm_all.append(mission_tm)
    mean_dir_all.append(mission_mean_dir)
    xs_gamma_loc_all.append(mission_xs_gamma_location)
    mission_break_depth_all.append(mission_break_depth)
    mission_time_all.append(mission_time)
    mission_spectra_all.append(mission_spectra.tolist())
    freq_all.append(freq.tolist())
    dir_bins_all.append(dirs.tolist())
    mission_dir_spectra_all.append(mission_dir_spectra.tolist())
    
    # Save mission number
    mission_number.append(int(mission_nc[41:-3]))

/opt/homebrew/Caskroom/mambaforge/base/envs/dunex-venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:784: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [3]:
mission_df = pd.DataFrame(cftime.num2pydate(mission_time_all, units=mission_dataset['time'].units,
                          calendar=mission_dataset['time'].calendar), columns=['time'])
mission_df['mission number'] = mission_number
mission_df['Hs [m]'] = hs_all
mission_df['Tm [s]'] = tm_all
mission_df['Mean Dir [deg]'] = mean_dir_all
mission_df['cross shore gamma location [m]'] = xs_gamma_loc_all
mission_df['break depth'] = mission_break_depth_all
mission_df['freq [hz]'] = freq_all
mission_df['energy density [m^2\hz]'] = mission_spectra_all
mission_df['direction bins [deg]'] = dir_bins_all
mission_df['directional energy density [m^2/hz/deg]'] = mission_dir_spectra_all

mission_df

,time,mission number,Hs [m],Tm [s],Mean Dir [deg],cross shore gamma location [m],break depth,freq [hz],energy density [m^2\hz],direction bins [deg],directional energy density [m^2/hz/deg]
0,2021-10-29 14:33:54.998780,78,2.193778,6.520426,80.702428,547.208908,6.267937,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.06743334686967555, 0.0875728585812643, 0.24...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[0.00017234805272892118, 0.000180635062861256..."
1,2021-10-13 13:19:59.997600,29,1.173307,5.940471,63.342316,276.593544,3.352306,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.02870440033112516, 0.01928257405997609, 0.0...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[9.174015576718375e-05, 8.881670510163531e-05..."
2,2021-10-07 14:34:59.998800,9,1.295971,4.938903,67.277178,312.260030,3.702775,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.013540998759169172, 0.014919749907868013, 0...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[2.557561128924135e-05, 2.5664361601229757e-0..."
3,2021-10-15 16:19:29.998200,39,0.655418,6.956270,84.357259,211.344492,1.872622,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.010834759694994502, 0.0077490358208274455, ...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[2.3921338652144186e-05, 2.4268156266771257e-..."
4,2021-10-28 15:41:59.998080,68,1.526962,6.728406,61.077903,391.185747,4.362749,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.03779555385559649, 0.04079146372305993, 0.0...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[7.68100144341588e-05, 7.782986358506605e-05,..."
...,...,...,...,...,...,...,...,...,...,...,...
62,2021-10-29 13:05:59.999280,75,2.487298,6.568453,76.479342,638.135759,7.106566,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.10780270480606641, 0.0995278847817784, 0.23...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[0.0003470113151706755, 0.0003708729054778814..."
63,2021-10-11 13:21:29.997960,24,2.267426,5.765862,59.019001,566.670076,6.478361,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.05141357787839604, 0.08320280224703631, 0.1...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[0.00013048572873231024, 0.000130322252516634..."
64,2021-10-05 18:18:29.998680,4,0.590386,8.364241,81.631654,200.441240,1.686817,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.00815565533744278, 0.010602991464714164, 0....","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[1.812473601603415e-05, 1.8152162738260813e-0..."
65,2021-10-14 13:07:29.998200,34,0.876134,6.759128,72.869430,237.838341,2.503241,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.018558775622316583, 0.01621167310050603, 0....","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[4.345577326603234e-05, 4.269385681254789e-05..."


In [4]:
mission_df.to_csv('../microSWIFT_data/mission_df_6mawac.csv')